In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=10, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=4, min_lr=1e-6,
    ),
]

In [4]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=10,
    overwrite=False,
)
     
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Reloading Tuner from trials/hyperband/tuner0.json


In [8]:
tuner.results_summary(num_trials=2)

Results summary
Results in trials/hyperband
Showing 2 best trials
Objective(name="val_weightedf2score", direction="max")

Trial 0054 summary
Hyperparameters:
batch_size: 32
class_weight: True
training_years: 2017_2018_2019
filters_power: 7
kernel_regularizer: l1l2
kernel_size: 3
activation: leaky_relu
pool_size: 2
spatial_dropout: 0.1
units_power: 8
dropout: 0.1
output_bias: False
dropout_rate: 0.0001
loss: binary_focal_crossentropy
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0053
Score: 0.6631672978401184

Trial 0084 summary
Hyperparameters:
batch_size: 64
class_weight: True
training_years: 2017_2018_2019
filters_power: 6
kernel_regularizer: l1l2
kernel_size: 3
activation: relu
pool_size: 4
spatial_dropout: 0.2
units_power: 8
dropout: 0.1
output_bias: True
dropout_rate: 0.001
loss: binary_focal_crossentropy
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0083
Score: 0.6603403687477112


In [6]:
%load_ext tensorboard
tb_path = str(trials_dir.joinpath('board'))
%tensorboard --logdir "$tb_path"